In [3]:
!pip install tensorflow numpy pandas scikit-learn tensorflow-datasets
import tensorflow as tf


In [4]:
tf.constant([[1.,2.,3.],[4.,5.,6.]], dtype=tf.float32)

2025-07-19 23:26:31.553282: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
tf.constant(44, dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=44.0>

In [6]:
tf.square(tf.constant([[1., 2., 3.], [4., 5., 6.]]))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [7]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [8]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
t@tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load a real-life dataset: California Housing (predict house prices)
housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target

# Split the data
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_housing, y_housing, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_h = scaler.fit_transform(X_train_h)
X_test_h = scaler.transform(X_test_h)

# Build a deep neural network for regression
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_h.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output: predicted house price
])



/home/manas/Documents/webdev/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
model.compile(optimizer='adam', loss=huber_fn, metrics=['mae'])



In [12]:
# Train the model
history = model.fit(X_train_h, y_train_h, epochs=30, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate on test set
test_loss, test_mae = model.evaluate(X_test_h, y_test_h)
print(f"Test Hubber loss: {test_mae:.2f}")

# Predict house price for a new sample
sample = X_test_h[0].reshape(1, -1)
predicted_price = model.predict(sample)[0][0]
print(f"Predicted house price: ${predicted_price * 100000:.2f}")

Epoch 1/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3700 - mae: 0.6875 - val_loss: 0.1774 - val_mae: 0.4584
Epoch 2/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1651 - mae: 0.4330 - val_loss: 0.1655 - val_mae: 0.4349
Epoch 3/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1469 - mae: 0.4020 - val_loss: 0.1561 - val_mae: 0.4176
Epoch 4/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1406 - mae: 0.3875 - val_loss: 0.1412 - val_mae: 0.3910
Epoch 5/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1353 - mae: 0.3816 - val_loss: 0.1401 - val_mae: 0.3818
Epoch 6/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1312 - mae: 0.3687 - val_loss: 0.1357 - val_mae: 0.3764
Epoch 7/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1251 - mae: 0.3606 - val_loss: 0.1432 - val_mae: 0.3766
Epoch 8/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1228 - mae: 0.3536 - val_loss: 0.1549 - val_mae: 0.4252
Epoch 9/30
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [13]:
from tensorflow import keras
# Save the model with a custom loss function
model.save("my_model_with_a_custom_loss.h5")
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
custom_objects={"huber_fn": huber_fn})

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,035 (109.52 KB)

 Trainable params: 28,033 (109.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [15]:
model.predict(sample)[0][0]*100000


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


np.float32(51131.562)

In [16]:
@tf.function
def tf_cube(x):
    return x ** 3

In [17]:
tf_cube.python_function(2)

8

In [18]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

In [19]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times

In [20]:
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)

In [21]:
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


2025-07-19 23:26:53.869853: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
with tf.io.TFRecordWriter("data.tfrecord") as writer:
    writer.write("My name is manas")
    writer.write("I study in PICT")


In [23]:
file_path = "data.tfrecord"
raw_dataset = tf.data.TFRecordDataset(file_path)
for item in raw_dataset:
    print(item)

tf.Tensor(b'My name is manas', shape=(), dtype=string)
tf.Tensor(b'I study in PICT', shape=(), dtype=string)


2025-07-19 23:26:53.905177: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:381] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-07-19 23:26:53.905975: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
#loading complressed files
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Compressed data example")
    f.write(b"Another compressed data example")
datset = tf.data.TFRecordDataset("my_compressed.tfrecord", compression_type="GZIP")
for item in datset:
    print(item)

tf.Tensor(b'Compressed data example', shape=(), dtype=string)
tf.Tensor(b'Another compressed data example', shape=(), dtype=string)


In [25]:
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)
table

In [26]:
categorical_data = tf.constant(["INLAND", "NEAR OCEAN", "ISLAND", "UNKNOWN"])
cat_indices = table.lookup(categorical_data)
print(cat_indices.numpy())
cat_one_hot  = tf.one_hot(cat_indices,depth = len(vocab)+num_oov_buckets)
print(cat_one_hot.numpy())

[1 2 4 5]
[[0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [27]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)

In [28]:
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.5910189 , 0.22021341],
       [0.20604026, 0.25875568],
       [0.1538744 , 0.4310733 ],
       [0.56827116, 0.657869  ],
       [0.3946972 , 0.10202622],
       [0.24898303, 0.78690934],
       [0.6916238 , 0.6304847 ]], dtype=float32)>

In [29]:
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_embeddings = tf.nn.embedding_lookup(embedding_matrix, cat_indices)
print(cat_embeddings.numpy())

[[0.56827116 0.657869  ]
 [0.24898303 0.78690934]
 [0.20604026 0.25875568]
 [0.20604026 0.25875568]]


In [30]:
from tensorflow import keras

embedding_dim = 2  # Make sure this matches your earlier definition
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
num_oov_buckets = 2

# Inputs
regular_inputs = keras.layers.Input(shape=(8,))
categories = keras.layers.Input(shape=(), dtype=tf.string)

# Lookup indices for categories (specify output_shape)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, tf.range(len(vocab), dtype=tf.int64))
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats), output_shape=(1,), dtype=tf.int64)(categories)

# Embedding for categorical features
cat_embed = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets, output_dim=embedding_dim)(cat_indices)
cat_embed_flat = keras.layers.Flatten()(cat_embed)

# Concatenate regular and embedded categorical inputs
encoded_inputs = keras.layers.Concatenate()([regular_inputs, cat_embed_flat])

# Output layer
outputs = keras.layers.Dense(1)(encoded_inputs)

# Model
model = keras.models.Model(inputs=[regular_inputs, categories], outputs=outputs)

In [31]:
from tensorflow import keras
import numpy as np

# Example data (replace with your actual data)
X_train_h = np.random.rand(100, 8)  # 100 samples, 8 features

# Normalization layer
normalization = keras.layers.Normalization()

# Discretization layer (example bin boundaries, replace [...] with actual values)
bin_boundaries = [0.2, 0.4, 0.6, 0.8]  # Example boundaries for demonstration
discretization = keras.layers.Discretization(bin_boundaries)

# Build preprocessing pipeline
pipeline = keras.Sequential([normalization, discretization])

# Adapt normalization layer to your data
normalization.adapt(X_train_h)

# Adapt discretization layer to your data (optional, only needed if you want to use adapt)
# discretization.adapt(X_train_h)  # Uncomment if you want to adapt discretization

# Example: Transform data using the pipeline
X_transformed = pipeline(X_train_h)
print(X_transformed.numpy())

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 0 0]
 [3 0 0 4 0 4 0 4]
 [0 2 4 0 4 0 4 4]
 [0 4 0 0 0 3 4 2]
 [0 3 4 0 0 0 3 0]
 [0 0 2 4 4 4 4 3]
 [4 0 3 0 1 0 4 0]
 [4 0 0 0 4 0 0 0]
 [0 0 4 0 0 4 0 4]
 [2 0 4 0 0 0 0 0]
 [0 4 2 0 0 0 0 0]
 [0 0 0 0 2 0 0 4]
 [4 3 1 0 2 0 0 0]
 [0 4 0 0 4 1 0 4]
 [0 0 4 0 0 0 0 0]
 [0 0 0 0 0 3 0 2]
 [0 0 0 0 0 0 0 0]
 [4 2 4 0 1 0 0 0]
 [0 4 2 0 0 0 0 0]
 [4 0 0 3 0 0 4 0]
 [4 4 0 4 4 4 2 4]
 [0 0 0 0 4 0 2 0]
 [0 4 1 4 4 4 0 4]
 [0 0 4 0 0 4 4 4]
 [0 4 0 0 0 0 0 0]
 [4 0 0 0 0 4 4 0]
 [0 0 4 0 0 0 0 0]
 [4 0 0 4 0 0 0 4]
 [2 4 0 4 3 0 3 0]
 [0 0 0 3 1 4 4 4]
 [4 4 0 0 0 4 0 0]
 [0 0 3 2 1 0 4 0]
 [3 0 1 4 0 0 0 0]
 [4 0 3 0 0 4 4 0]
 [0 0 4 0 4 3 0 2]
 [0 0 2 2 4 0 4 0]
 [2 1 0 0 0 0 1 0]
 [4 4 0 0 0 4 4 2]
 [0 0 0 4 4 4 0 3]
 [2 3 0 0 0 2 4 0]
 [0 4 2 0 4 0 0 0]
 [0 0 3 0 0 0 0 4]
 [3 0 0 0 0 0 4 1]
 [1 4 4 4 0 0 0 0]
 [0 0 4 3 4 4 4 1]
 [0 1 4 4 0 0 0 0]
 [1 1 4 4 0 4 0 0]
 [0 0 0 0 0 0 0 1]
 [0 4 4 0 0 4 4 4]
 [0 0 0 2 0 0 0 4]
 [4 4 0 0 4 0 4 1]
 [0 2 4 4 2 

In [32]:
from tensorflow import keras

# Download MNIST directly from Keras (no tfds needed)
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

mnist_train = (X_train, y_train)
mnist_test = (X_test, y_test)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (60000, 28, 28) (60000,)
Test shape: (10000, 28, 28) (10000,)


In [33]:
import tensorflow_datasets as tfds
dataset = tfds.load(name="mnist")
mnist_train, mnist_test = dataset["train"], dataset["test"]

2025-07-19 23:27:00.690826: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [34]:
mnist_train = mnist_train.shuffle(10000).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)

In [35]:
import tensorflow as tf
from tensorflow import keras
# Normalize and expand dims for channel
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

mnist_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32).prefetch(1)
mnist_test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(1)


model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28, 28, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, 3, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


/home/manas/Documents/webdev/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.fit(mnist_train, epochs=10, validation_data=mnist_test)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 65s 34ms/step - accuracy: 0.9164 - loss: 0.2833 - val_accuracy: 0.9778 - val_loss: 0.0727
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 61s 32ms/step - accuracy: 0.9745 - loss: 0.0850 - val_accuracy: 0.9880 - val_loss: 0.0351
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 60s 32ms/step - accuracy: 0.9817 - loss: 0.0608 - val_accuracy: 0.9849 - val_loss: 0.0415
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 55s 30ms/step - accuracy: 0.9843 - loss: 0.0509 - val_accuracy: 0.9896 - val_loss: 0.0321
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9875 - loss: 0.0380 - val_accuracy: 0.9892 - val_loss: 0.0368
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9890 - loss: 0.0347 - val_accuracy: 0.9911 - val_loss: 0.0304
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9902 - loss: 0.0321 - val_accuracy: 0.9919 - val_loss: 0.0280
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 66s 35ms/step - accuracy: 0.9909 -

In [37]:
test_loss, test_acc = model.evaluate(mnist_test)
print("Test accuracy:", test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9900 - loss: 0.0336
Test accuracy: 0.9919999837875366


In [41]:
from sklearn.datasets import load_sample_image

def load_image(name):
    """
    Loads a sample image from sklearn and normalizes it to [0, 1].
    Args:
        name (str): The filename, e.g., "china.jpg" or "flower.jpg"
    Returns:
        np.ndarray: Normalized image array
    """
    img = load_sample_image(name)
    return img / 255.0




In [51]:
!pip install PIL
from sklearn.datasets import load_sample_image
# Load sample images
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


ImportError: The Python Imaging Library (PIL) is required to load data from jpeg files. Please refer to https://pillow.readthedocs.io/en/stable/installation.html for installing PIL.